FUTURO CAMBIAR A 4 res layer en gen_DOWN

In [ ]:
pip install comet_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.9/514.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 41.4 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.5.1
    Uninstalling websocket-client-1.5.1:
      Successfully uninstalled websocket-client-1.5.1


In [ ]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
experiment = Experiment(
  api_key = "3xZI0xXZ4oGsYBUUQ5FzbVxCC",
  project_name = "AWasserstein Loss esta vez bien",
  workspace="manuelnevado",
  log_code=True,
  log_graph=True,
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelnevado/awasserstein-loss-esta-vez-bien/a8cc95e23b3e4b22b4bf1dfc8e1e548b



In [ ]:
experiment.add_tag("Entrenamiento10, viejos a bebes")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf Adults && rm -rf Kids

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/TFG/UTK_Corto.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [ ]:
import os
import numpy as np
import math
import sys
from time import time
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import log2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torchvision import transforms, utils
from torch.utils.tensorboard import SummaryWriter



from torch.utils.data import Dataset, DataLoader, random_split
import os
import pandas as pd
from tqdm import tqdm
import logging
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from math import log2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
LEARNING_RATE_GEN = 8e-5
LEARNING_RATE_DIS = 1e-4
LAMBDA_IDENTITY = 0
LAMBDA_CYCLE = 10
NUM_WORKERS = 2
NUM_EPOCHS = 1500
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_GEN_H = "genh.pth.tar"
CHECKPOINT_GEN_Z = "genz.pth.tar"
CHECKPOINT_CRITIC_H = "critich.pth.tar"
CHECKPOINT_CRITIC_Z = "criticz.pth.tar"

hyper_params = {
   "learning_rate_gen": LEARNING_RATE_GEN,
   "learning_rate_dis": LEARNING_RATE_DIS,
   "EPOCH": NUM_EPOCHS,
   "batch_size": BATCH_SIZE,
   "lambda_identity": LAMBDA_IDENTITY
}

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    img = img.to("cpu")
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        return npimg
    else:
        return np.transpose(npimg, (1, 2, 0))

In [ ]:
class test_class(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Conv2d(3, 3 * 2, kernel_size=3, padding=1)
    def forward(self,x):
        return self.layer(x)


def test():  
    model = test_class()
    x = torch.randn((2,3,16,16))
    print(model(x).shape)
test()

torch.Size([2, 6, 16, 16])


In [ ]:
class samplingLayer(nn.Module):

    def __init__(self, init_channels=3, final_channels=3):
        super().__init__()
        self.init_channels = init_channels
        self.final_channels = final_channels
        self.layer = nn.Conv2d(int(init_channels),int(final_channels),kernel_size=3,padding=1)
        self.norm = nn.InstanceNorm2d(final_channels)
        self.leaky = nn.ReLU(inplace=True)
        return

    def forward(self,x):
        x = self.layer(x)
        x = self.norm(x)
        x = self.leaky(x)
        return x

def test():
    model = samplingLayer(3,100)
    x = torch.randn((2,3,32,32))
    print(model(x).shape)
    return

test()

torch.Size([2, 100, 32, 32])


In [ ]:
class downSamplingLayer(nn.Module):
    
    def __init__(self, channels=3, channels_out=6):
        super().__init__()
        self.channels = channels
        self.channels_out = channels_out
        self.layer = nn.Conv2d(int(channels),channels_out,2,stride=2)
        #self.norm = nn.InstanceNorm2d(channels_out)
        self.leaky = nn.LeakyReLU(inplace=True)
        return
    
    def forward(self,x):
        x = self.layer(x)
        #x = self.norm(x)
        x = self.leaky(x)
        return x

    
def test():
    model = downSamplingLayer(3,9)
    x = torch.randn((2,3,32,32))
    print(model(x).shape)
    return

test()


torch.Size([2, 9, 16, 16])


In [ ]:
class upSamplingLayer(nn.Module):
    
    def __init__(self,channels=6,channels_out=3):
        super().__init__()
        self.channels = channels
        self.channels_out = 6
        self.layer = nn.ConvTranspose2d(int(channels),int(channels_out),2,stride=2)
        self.norm = nn.InstanceNorm2d(channels_out)
        self.leaky = nn.LeakyReLU(0.2)
        return
    
    def forward(self, x):
        x = self.layer(x)
        x = self.norm(x)
        x = self.leaky(x)
        return x
    
def test():
    model = upSamplingLayer(channels=10)
    x = torch.randn((2,10,8,8))
    print(model(x).shape)
    return

test()

torch.Size([2, 3, 16, 16])


In [ ]:
class residualLayer(nn.Module):
    
    def __init__(self, channels = 3):
        self.channels = channels
        super().__init__()
        layer1 = nn.Sequential(
            nn.Conv2d(self.channels, self.channels, kernel_size=3,padding=1),
            nn.InstanceNorm2d(self.channels),
            nn.ReLU(inplace=True)
        )
        layer2 = nn.Sequential(
            nn.Conv2d(self.channels, self.channels, kernel_size=3,padding=1),
            nn.InstanceNorm2d(self.channels),
            nn.Identity()
        ) 
        
        self.bifurcacion = nn.Sequential(
            layer1,
            layer2
        )
        return

    def forward(self,x):
      x = x + self.bifurcacion(x)
      return x
    
def test():
    model = residualLayer()
    x = torch.randn((2,3,16,16))
    print(model(x).shape)
    return

test()

torch.Size([2, 3, 16, 16])


In [ ]:
class DownSampler(nn.Module):
    
    def __init__(self, img_resolution, channels=3, frontier=8):
        #frontier parameter must be the same between UpSampler and DownSampler
        super().__init__()
        self.channels = channels
        self.frontier = frontier
        self.img_resolution = img_resolution
        
        
        self.model = nn.Sequential()
        if self.img_resolution == self.frontier:
            self.model.append(samplingLayer(channels))
        else:
            while img_resolution > self.frontier:
                self.model.append(downSamplingLayer(channels,channels))
                img_resolution = img_resolution/2
        return
    
    def crece(self):
        #self.model.insert(0,downSamplingLayer(self.channels))
        model_new = nn.Sequential()
        model_new.append(downSamplingLayer(self.channels,self.channels))
        for layer in self.model:
            model_new.append(layer)
        self.model = model_new
        return
    
    def forward(self,x):
        return self.model(x)
    
def test():
    img = 8
    model = DownSampler(img,channels=1)
    
    for _ in range(2):
        x = torch.randn((5,1,img,img))
        print(f"model: {model}")
        print(x.shape)
        print(model(x).shape)
        img *=2
        model.crece()
    return

test()

model: DownSampler(
  (model): Sequential(
    (0): samplingLayer(
      (layer): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(3, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (leaky): ReLU(inplace=True)
    )
  )
)
torch.Size([5, 1, 8, 8])
torch.Size([5, 3, 8, 8])
model: DownSampler(
  (model): Sequential(
    (0): downSamplingLayer(
      (layer): Conv2d(1, 1, kernel_size=(2, 2), stride=(2, 2))
      (leaky): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (1): samplingLayer(
      (layer): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(3, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (leaky): ReLU(inplace=True)
    )
  )
)
torch.Size([5, 1, 16, 16])
torch.Size([5, 3, 8, 8])


In [ ]:
class ResidualSampler(nn.Module):
    
    def __init__(self, channels=3, n_residuals = 2):
        super().__init__()
        self.channels =channels
        self.n_residuals = n_residuals
        self.model = nn.Sequential()
        for _ in range(self.n_residuals):
            self.model.append(residualLayer(channels))
        return
    
    def crece(self, n=1):
        for _ in range(n):
            self.model.append(residualLayer(self.channels))
        return
    
    def forward(self,x):
        return self.model(x)

def test():
    img = 8
    model = ResidualSampler()
    for _ in range(3):
        x = torch.randn((5,3,img,img))
        print(model(x).shape)
        model.crece()
test()

torch.Size([5, 3, 8, 8])
torch.Size([5, 3, 8, 8])
torch.Size([5, 3, 8, 8])


In [ ]:
class UpSampler(nn.Module):
    
    def __init__(self,img_resolution, channels=3, frontier=8):
        #frontier parameter must be the same between UpSampler and DownSampler
        super().__init__()
        self.frontier = frontier
        self.img_resolution = img_resolution
        self.channels = channels
        self.model = nn.Sequential() 
        if self.img_resolution == self.frontier:
            self.model.append(samplingLayer(channels,final_channels=channels))
        else:
            while img_resolution > self.frontier:
                self.model.append(upSamplingLayer(channels,channels))
                img_resolution = img_resolution/2
        return
    
    def crece(self):
        self.model.append(upSamplingLayer(self.channels,self.channels))
    
    def forward(self,x):
        return self.model(x)

def test():
    img = 8
    model = UpSampler(img,channels=1)
    
    for _ in range(5):
        x = torch.randn((5,1,8,8))
        print(f"vector size: {x.shape}")
        print(model(x).shape)
        img *=2
        model.crece()
    return

test()

vector size: torch.Size([5, 1, 8, 8])
torch.Size([5, 1, 8, 8])
vector size: torch.Size([5, 1, 8, 8])
torch.Size([5, 1, 16, 16])
vector size: torch.Size([5, 1, 8, 8])
torch.Size([5, 1, 32, 32])
vector size: torch.Size([5, 1, 8, 8])
torch.Size([5, 1, 64, 64])
vector size: torch.Size([5, 1, 8, 8])
torch.Size([5, 1, 128, 128])


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), # Pads the input tensor using the reflection of the input boundary
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features), 
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features)
        )

    def forward(self, x):
        return x + self.block(x)



class Generador_down(nn.Module):
    
    def __init__(self, img_resolution, frontier, num_residuals=3, init_channels=1, frontier_channels=100, paso_de_canal=10, rabit_hole=5):
        super().__init__()
        self.img_resolution = img_resolution
        self.frontier = frontier
        self.growing_step = int(log2(img_resolution)-log2(frontier))
        self.init_channels = init_channels
        self.final_channels = 2**self.growing_step*self.init_channels
        self.frontier_channels = frontier_channels
        self.paso_de_canal = paso_de_canal

        #Inicial
        out_features = 64
        self.model = [
            #nn.ReflectionPad2d(init_channels),
            nn.Conv2d(init_channels, out_features, kernel_size=3,stride=1,padding=1),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True)
        ]
        in_features = out_features
        
        current_size = img_resolution
        #DOWNSAMPLING
        while current_size>frontier:
          out_features *= 2
          self.model += [
              nn.Conv2d(in_features, out_features, 3, stride=2, padding=1), # --> width/2 ,heigth/2
              nn.InstanceNorm2d(out_features),
              nn.ReLU(inplace=True)
          ]
          in_features = out_features
          current_size = int(current_size/2)
          #print(f"current_size: {current_size}")
        
        #RESIDUAL
        for _ in range(num_residuals):
            self.model += [ResidualBlock(out_features)]

        #UPSAMPLING
        for _ in range(rabit_hole):
            out_features //= 2
            self.model += [
                nn.Upsample(scale_factor=2) if current_size < frontier else nn.Identity(), # --> width*2, heigh*2
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            current_size *= 2
        
        # Output Layer
        self.model += [
                  #nn.ReflectionPad2d(init_channels),
                  nn.Conv2d(out_features, init_channels, 3,stride=1,padding=1),
                  nn.Tanh()
                 ]
        
        #UNPACK
        self.model_sequential = nn.Sequential(*self.model)
        

        return
    
    def crece(self):
        if self.growing_step > 0:
            
            #Crece Down
            self.growing_step -= 1
            new_channels = 2**self.growing_step*self.init_channels
            
            #La primera
            self.initial = downSamplingLayer(self.init_channels,new_channels)
            
            #La nueva bajada
            new_down = downSamplingLayer(new_channels,new_channels*2)
            self.down_layers.insert(0,new_down)
            
            #La nueva subida
            new_up = samplingLayer(new_channels*2,new_channels)
            self.up_layers.append(new_up)
            
            #La ultima capa
            self.last_layer = samplingLayer(new_channels,self.init_channels)
            return True
            
        else:
            print("La red no puede crecer mas")
        
        return False
    
    def crece_res(self, times=1):
        
        for _ in range(times):
            self.res_layers.append(residualLayer(self.final_channels))
    
    def forward(self,x):
      #print(f"init shape{x.shape}")
      for layer in self.model:
        x = layer(x)
        #print(f"for layer:{layer} shape returned is: {x.shape}")
      return x
      #return self.model_sequential(x)

def test():
    x = torch.randn((2,3,512,512)).to("cuda")
    model = Generador_down(512,64,init_channels=3, paso_de_canal=10).to("cuda")
    print(model)
    print(f"shape returned by model: {model(x).shape}")    
    return

test()
        

Generador_down(
  (model_sequential): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (8): ReLU(inplace=True)
    (9): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (11): ReLU(inplace=True)
    (12): ResidualBlock(
      (block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(512, 512, kernel_size=(3, 3), stride=(

In [ ]:
class Generador_res(nn.Module):
    
    def __init__(self, img_resolution, frontier, num_de_paso=2, init_channels=1, frontier_channels=100, paso_de_canal=10, num_residuals=5):
        super().__init__()
        self.img_resolution = img_resolution
        self.frontier = frontier
        self.growing_step = int(log2(img_resolution)-log2(frontier))
        self.init_channels = init_channels
        self.final_channels = 2**self.growing_step*self.init_channels
        self.frontier_channels = frontier_channels
        self.paso_de_canal = paso_de_canal
  
        #construccion del modelo Res
        #Subida de canales
        self.res_up = nn.Sequential()
        current_channels = self.init_channels
        for _ in range(num_de_paso):
          self.res_up.append(samplingLayer(current_channels, current_channels*2))
          current_channels *= 2
        
        while current_channels+paso_de_canal < frontier_channels:
          self.res_up.append(samplingLayer(current_channels,current_channels+paso_de_canal))
          current_channels+=paso_de_canal
        self.res_up.append(samplingLayer(current_channels,frontier_channels))
        
        #residuales
        self.res_block = nn.Sequential()
        for _ in range(num_residuals):
          self.res_block.append(residualLayer(frontier_channels))
        
        #para arriba
        self.res_down = nn.Sequential()
        current_channels = frontier_channels
        while current_channels-paso_de_canal > init_channels:
          self.res_down.append(samplingLayer(current_channels, current_channels-paso_de_canal))
          current_channels = current_channels-paso_de_canal
        self.res_down.append(samplingLayer(current_channels,init_channels))

        return
    
    def crece(self):
        if self.growing_step > 0:
            print("Growing deprecated")
            return True
            
        else:
            print("La red no puede crecer mas")
        
        return False
    
    def crece_res(self, times=1):
        
        """for _ in range(times):
            self.res_layers.append(residualLayer(self.final_channels))
        """
        print("Crece res deprecated")
        return

    def forward(self,x):
        
        #print(f"shape initial_out: {x.shape}")
        for layer in self.res_up:
            x = layer(x)
            #print(f"shape down_out: {x.shape}")
        
        #for layer in self.res_block:
        #    x = layer(x)
            #print(f"shape res_out: {x.shape}")
        
        for layer in self.res_down:
            x = layer(x)
            #print(f"shape up_out: {x.shape}")
                    
        return x

def test():
    x = torch.randn((5,3,64,64)).to("cuda")
    model = Generador_res(64,64, num_de_paso=4, init_channels=3).to("cuda")
    print(model)
    print(f"shape returned by model: {model(x).shape}")
    
    return

test()

Generador_res(
  (res_up): Sequential(
    (0): samplingLayer(
      (layer): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(6, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (leaky): ReLU(inplace=True)
    )
    (1): samplingLayer(
      (layer): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(12, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (leaky): ReLU(inplace=True)
    )
    (2): samplingLayer(
      (layer): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(24, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (leaky): ReLU(inplace=True)
    )
    (3): samplingLayer(
      (layer): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm): InstanceNorm2d(48, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (leaky): ReLU(inplace=T

In [ ]:
class Discriminador(nn.Module):
    def __init__(self, img_size,channels =3, frontier=4, paso_de_canal=2):
        super().__init__()
        self.channels = channels
        self.img_size = img_size
        self.frontier = frontier
        self.paso_de_canal = paso_de_canal
        nc = channels
        ndf = 64
        self.layers = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(nc,ndf,4,2,1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 64 x 64
            nn.Conv2d(ndf,ndf*2,4,2,1, bias=False),
            nn.InstanceNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 32 x 32
            nn.Conv2d(ndf*2, ndf * 4, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            #ResidualBlock(ndf*4),
            #ResidualBlock(ndf*4),

            # state size. (ndf*4) x 16 x 16
            nn.Conv2d(ndf*4,ndf*8,4,1,1),
            nn.InstanceNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 15 x 15
            nn.Conv2d(ndf*8,1,4,1,1)
            # state size. 1 x 14 x 14
        )
        
        """#Las capas de crecimiento
        output_features = 32
        input_features = channels
        i = 0
        while img_size > self.frontier:
            self.layers.append(nn.Conv2d(
                input_features,output_features,
                kernel_size=3,padding=1,stride=1, padding_mode='reflect'))
            if i>0:
              self.layers.append(nn.InstanceNorm2d(output_features))
            self.layers.append(nn.LeakyReLU(0.2, inplace=True))
            img_size = img_size/2
            input_features = output_features
            output_features *= 2

        
        # La ultima capa
        self.layers.append(nn.Conv2d(input_features,1,
                                        kernel_size=4,
                                        stride=1,
                                        padding=1,
                                        padding_mode="reflect",))"""
        
        return
    
    def crece(self):
        print("Growing Deprecated")
        '''self.img_size *= 2
        self.layers.insert(0,nn.Conv2d(3,3,1,stride=2))'''
        return
    
    def forward(self, x):
      #print(f"input shape: {x.shape}")
      return self.layers(x)

def test(i):
    img_size = i
    model = Discriminador(img_size, frontier=2,channels=3).to(DEVICE)
    
    print(f"img_size: {img_size}")
    x = torch.randn((1,3,img_size,img_size)).to(DEVICE)

    print(model(x))
    print(torch.mean(model(x)))
    print(model(x).shape)
    print(model)
    return

test(64)
    

img_size: 64
tensor([[[[-0.1005, -0.0104,  0.8960,  0.3281, -0.0969, -0.2686],
          [ 0.0905,  0.4450,  0.3765,  0.5920, -0.1397,  0.0278],
          [ 0.2782,  0.3142,  0.8912,  0.6288,  0.5065, -0.3603],
          [ 0.1740, -0.4275,  0.2641,  0.8438,  0.0948, -0.6652],
          [ 0.2197, -0.5648,  0.2173, -0.2449,  0.4499, -0.1673],
          [ 0.8336,  0.1766, -0.1296,  0.0607, -0.2620,  0.3027]]]],
       device='cuda:0', grad_fn=<ConvolutionBackward0>)
tensor(0.1548, device='cuda:0', grad_fn=<MeanBackward0>)
torch.Size([1, 1, 6, 6])
Discriminador(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_siz

In [ ]:
transforms1 = A.Compose(
    [
        A.Resize(width=32, height=32),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5,], std=[0.5], max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"},
)

transforms2 = A.Compose(
    [
        A.Resize(width=16, height=16),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[ 0.5], std=[0.5], max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"},
)

In [ ]:
from PIL import Image
import os
from torch.utils.data import Dataset
import numpy as np

class TFGAN_Dataset(Dataset):
    
    #Por estandar
    #  root_A = Adultos
    #  root_b = Kids
 
    def __init__(self, root_A, root_B, transform1=None, transform2=None, gray_scale=True):    
        self.root_A = root_A
        self.root_B = root_B
        self.transform1 = transform1
        self.transform2 = transform2
        self.gray_scale = gray_scale
        
        self.root_A_images = os.listdir(root_A)
        self.root_B_images = os.listdir(root_B)
        self.length_dataset = max(len(self.root_A_images), len(self.root_B_images))
        self.root_A_len = len(self.root_A_images)
        self.root_B_len = len(self.root_B_images)
        
    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        
        A_image = self.root_A_images[index % self.root_A_len]
        B_image = self.root_B_images[index % self.root_B_len]

        A_path = os.path.join(self.root_A, A_image)
        B_path = os.path.join(self.root_B, B_image)

        A_image = np.array(Image.open(A_path).convert("L" if self.gray_scale else "RGB"))
        B_image = np.array(Image.open(B_path).convert("L" if self.gray_scale else "RGB"))
        
        A_image1 = A_image
        A_image2 = A_image
        
        B_image1 = B_image
        B_image2 = B_image
        
        if self.transform1:
            augmentations1 = self.transform1(image=A_image, image0=B_image)
            augmentations2 = self.transform2(image=A_image, image0=B_image)

            A_image1 = augmentations1["image"]
            A_image2 = augmentations2["image"]
            B_image1 = augmentations1["image0"]
            B_image2 = augmentations2["image0"]
            

        return A_image1, B_image1, A_image2, B_image2

def test():
    train_path = "."
    Adults_train = os.path.join(train_path, 'Adults')
    Kids_train = os.path.join(train_path, 'Kids')
    
    data = TFGAN_Dataset(Adults_train, Kids_train, transform1=transforms1, transform2=transforms2, gray_scale=True)
    
    model = ResidualSampler(channels=1)
    print(model(data[0][0]).shape)
    print(data[0][2].shape)
    return True
test()

torch.Size([1, 32, 32])
torch.Size([1, 16, 16])


True

In [ ]:
def loss_gan(fake,real):
    MSE = nn.L1Loss()
    return MSE(fake,real)

def loss_dis(fake, real):
    MSE = nn.MSELoss()
    return MSE(real,fake)

def loss_id(in1, out1, in2, out2):
    return torch.abs(in1-out1).mean() + torch.abs(in2-out2).mean()

def dis_loss_wasserstein_gan(fake_img, real_img):
    return torch.mean(real_img)-torch.mean(fake_img)

def gen_loss_wasserstein_gan(fake_img): #fake_img es gen(input)
    return torch.mean(fake_img)


In [ ]:
### Checkpoints
import random, torch, os, numpy as np
import copy

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    #print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
import torch
import sys
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image

def train_individual(
    writer, epoch, i,
    loader, l1, mse, 
    disc_Adults_down, disc_Kids_down, down_sampler, opt_disc_down, opt_gen_down, d_down_scaler, g_down_scaler,
    disc_Adults_res, disc_Kids_res, res_sampler, opt_disc_res, opt_gen_res, d_res_scaler, g_res_scaler,
    disc_Adult_up, disc_Kids_up, up_sampler, opt_disc_up, opt_gen_up, d_up_scaler, g_up_scaler   
):
    
    adults_reals = 0
    adults_fakes = 0
    
    loop = tqdm(loader, leave=True)
    
    for idx, (adult, kid, adult_frontier, kid_frontier) in enumerate(loop):
        #Mandamos las cosas a su sitio
        adult = adult.to(DEVICE)
        adult_frontier = adult_frontier.to(DEVICE)        
        kid = kid.to(DEVICE)
        kid_frontier = kid_frontier.to(DEVICE)
        
        #Entrenamos los discriminadores
        with torch.cuda.amp.autocast():
            fake_adult_down = down_sampler(kid)
            fake_adult_res = res_sampler(kid_frontier)
            fake_adult_up = up_sampler(kid_frontier)
            
            #DOWN
            disc_adult_down_real = disc_Kids_down(adult_frontier)
            disc_adult_down_fake = disc_Kids_down(fake_adult_down.detach())
            
            disc_adult_down_real_loss = mse(
                disc_adult_down_real, 
                torch.ones_like(disc_adult_down_real)
            )
            disc_adult_down_fake_loss = mse(
                disc_adult_down_fake,
                torch.zeros_like(disc_adult_down_fake)
            )
            disc_adult_down_loss = disc_adult_down_real_loss + disc_adult_down_fake_loss
            writer.add_scalar("Down_dis_loss/train", disc_adult_down_loss,  i)
            
            #RES
            disc_adult_res_real = disc_Kids_res(adult_frontier)
            disc_adult_res_fake = disc_Kids_res(fake_adult_res.detach())
            
            adults_reals += disc_adult_res_real.mean().item()
            adults_fakes += disc_adult_res_fake.mean().item()
            
            disc_adult_res_real_loss = mse(
                disc_adult_res_real, 
                torch.ones_like(disc_adult_res_real)
            )
            disc_adult_res_fake_loss = mse(
                disc_adult_res_fake,
                torch.zeros_like(disc_adult_res_fake)
            )
            disc_adult_res_loss = disc_adult_res_real_loss + disc_adult_res_fake_loss
            writer.add_scalar("Res_dis_loss/train", disc_adult_res_loss,  i)
            
            #UP
            disc_adult_up_real = disc_Kids_up(adult)
            disc_adult_up_fake = disc_Kids_up(fake_adult_up.detach())
            
            adults_reals += disc_adult_up_real.mean().item()
            adults_fakes += disc_adult_up_fake.mean().item()
            
            disc_adult_up_real_loss = mse(
                disc_adult_up_real, 
                torch.ones_like(disc_adult_up_real)
            )
            disc_adult_up_fake_loss = mse(
                disc_adult_up_fake,
                torch.zeros_like(disc_adult_up_fake)
            )
            disc_adult_up_loss = disc_adult_up_real_loss + disc_adult_up_fake_loss
            writer.add_scalar("Up_dis_loss/train", disc_adult_up_loss, i)
            
            #Como no hay consistencia de ciclo no neecesitamos entrenar los segundos generadores 
            #Lo juntamos todo, no nos sirve para entreno solo para ver como va la cosa
            D_loss = (disc_adult_down_loss + disc_adult_res_loss + disc_adult_up_loss)/3
            
            adults_reals += disc_adult_up_real.mean().item()
            adults_fakes += disc_adult_up_fake.mean().item()
        
        
        #APLICAMOS LOS CAMBIOS DEL ENTRENO:
        
        #DOWN
        opt_disc_down.zero_grad()
        d_down_scaler.scale(disc_adult_down_loss).backward()
        d_down_scaler.step(opt_disc_down)
        d_down_scaler.update()
        #RES
        opt_disc_res.zero_grad()
        d_res_scaler.scale(disc_adult_res_loss).backward()
        d_res_scaler.step(opt_disc_res)
        d_res_scaler.update()
        #UP
        opt_disc_up.zero_grad()
        d_up_scaler.scale(disc_adult_up_loss).backward()
        d_up_scaler.step(opt_disc_up)
        d_up_scaler.update()
        
        
        
        #Ahora entrenamos los generadores con lo que hemos sacado de los discriminadores
        with torch.cuda.amp.autocast():
            #adversarial loss para los generadores
            dis_adult_down_fake = disc_Adults_down(fake_adult_down)
            loss_gen_down_adult = mse(dis_adult_down_fake, torch.ones_like(dis_adult_down_fake))
            writer.add_scalar("Down_gen_loss/train", loss_gen_down_adult, i)
            
            dis_adult_res_fake = disc_Adults_res(fake_adult_res)
            loss_gen_res_adult = mse(dis_adult_res_fake, torch.ones_like(dis_adult_res_fake))
            writer.add_scalar("Res_gen_loss/train", loss_gen_res_adult, i)
            
            dis_adult_up_fake = disc_Adult_up(fake_adult_up)
            loss_gen_up_adult = mse(dis_adult_up_fake, torch.ones_like(dis_adult_up_fake))
            writer.add_scalar("Up_gen_loss/train", loss_gen_up_adult, i)
            
            #identity loss (para no alterar las caracteristicas principales)
            identity_adult_down = down_sampler(adult)
            identity_adult_down_loss = l1(adult_frontier, identity_adult_down)
            writer.add_scalar("Identity_down_loss/train", identity_adult_down_loss, i)
            
            identity_adult_res = res_sampler(adult_frontier)
            identity_adult_res_loss = l1(adult_frontier, identity_adult_res)
            
            identity_adult_up = up_sampler(adult_frontier)
            identity_adult_up_loss = l1(adult, identity_adult_up)
            
            #Se suma todo
            g_down_loss = (loss_gen_down_adult + identity_adult_down_loss*LAMBDA_IDENTITY)
            g_res_loss = (loss_gen_res_adult + identity_adult_res_loss*LAMBDA_IDENTITY)
            g_up_loss = (loss_gen_up_adult + identity_adult_up_loss*LAMBDA_IDENTITY)
            
        #APLICAMOS LOS CAMBIOS DEL ENTRENO:
        
        #DOWN
        opt_gen_down.zero_grad()
        g_down_scaler.scale(g_down_loss).backward()
        g_down_scaler.step(opt_gen_down)
        g_down_scaler.update()
        
        #RES
        opt_gen_res.zero_grad()
        g_res_scaler.scale(g_res_loss).backward()
        g_res_scaler.step(opt_gen_res)
        g_res_scaler.update()
        
        #UP
        opt_gen_up.zero_grad()
        g_up_scaler.scale(g_up_loss).backward()
        g_up_scaler.step(opt_gen_up)
        g_up_scaler.update()
        
        if idx%500==0:
            save_image(fake_adult_down*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/Entrenamiento3/adult_down{idx}.png")
            save_image(fake_adult_res*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/Entrenamiento3/adult_res{idx}.png")
            save_image(fake_adult_up*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/Entrenamiento3/adult_up{idx}.png")
            save_image(up_sampler(res_sampler(down_sampler(kid)))*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/Entrenamiento3/fake_test{idx}.png")
            save_checkpoint(down_sampler, opt_gen_down, filename="/content/drive/MyDrive/TFG/Images/Entrenamiento3/Modelos/gen_down.pth.tar")
            save_checkpoint(disc_Kids_down, opt_disc_down, filename="/content/drive/MyDrive/TFG/Images/Entrenamiento3/Modelos/disc_Kids_down.pth.tar")
            save_checkpoint(disc_Adults_down, opt_disc_down, filename="/content/drive/MyDrive/TFG/Images/Entrenamiento3/Modelos/disc_Adults_down.pth.tar")
            save_checkpoint(res_sampler, opt_gen_res, filename="/content/drive/MyDrive/TFG/Images/Entrenamiento3/Modelos/gen_res.pth.tar")
            save_checkpoint(disc_Kids_res, opt_disc_res, filename="/content/drive/MyDrive/TFG/Images/Entrenamiento3/Modelos/disc_Kids_res.pth.tar")
            save_checkpoint(disc_Adults_res, opt_disc_res, filename="/content/drive/MyDrive/TFG/Images/Entrenamiento3/Modelos/disc_Adults_res.pth.tar")

            
            img_grid = torchvision.utils.make_grid(adult)
            #matplotlib_imshow(img_grid, one_channel=True)
            img_grid_kid = torchvision.utils.make_grid(fake_adult_down)
            writer.add_image('input', torchvision.utils.make_grid(kid_frontier), i)
            writer.add_image('output', torchvision.utils.make_grid(fake_adult_res), i)

            writer.add_image('input_down', torchvision.utils.make_grid(kid), i)
            writer.add_image('output_down', torchvision.utils.make_grid(fake_adult_down), i)

            writer.add_image('target', torchvision.utils.make_grid(adult_frontier), i)
            #writer.add_image('fake_adults_down', torchvision.utils.make_grid(fake_adult_down),idx)
            #writer.add_image('fake_adults_res', torchvision.utils.make_grid(fake_adult_res), idx)
            writer.add_graph(down_sampler,kid)
            writer.add_graph(disc_Adults_down,kid)
            writer.add_graph(res_sampler,kid_frontier)
        
        loop.set_postfix(adults_reals = adults_reals / (idx+1), adults_fakes = adults_fakes / (idx+1))
            
            

In [ ]:
import torch
import sys
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image

def train_individual_down(
    writer, epoch, i,
    loader, l1, mse, 
    disc_Adults_down, disc_Kids_down, down_sampler, opt_disc_Adults_down, opt_disc_Kids_down, opt_gen_down, d_down_scaler, g_down_scaler,
):
    
    adults_reals = 0
    adults_fakes = 0
    
    loop = tqdm(loader, leave=True)
    
    for idx, (adult, kid, adult_frontier, kid_frontier) in enumerate(loop):
        i+=1
        #Mandamos las cosas a su sitio
        adult = adult.to(DEVICE)
        adult_frontier = adult_frontier.to(DEVICE)        
        kid = kid.to(DEVICE)
        kid_frontier = kid_frontier.to(DEVICE)
        
        #Entrenamos los discriminadores
        with torch.cuda.amp.autocast():
            fake_adult_down = down_sampler(kid)
            
            #DOWN
            disc_adult_down_real = disc_Adults_down(adult_frontier)
            disc_adult_down_fake = disc_Adults_down(fake_adult_down.detach())
            
            """disc_adult_down_real_loss = mse(
                disc_adult_down_real, 
                torch.ones_like(disc_adult_down_real)
            )
            disc_adult_down_fake_loss = mse(
                disc_adult_down_fake,
                torch.zeros_like(disc_adult_down_fake)
            )
            
            
            disc_adult_down_loss = disc_adult_down_real_loss + disc_adult_down_fake_loss"""
            disc_adult_down_loss = torch.mean(disc_adult_down_real) - torch.mean(disc_adult_down_fake) 
            writer.add_scalar("DisLoss", disc_adult_down_loss,  i)

            
            
            
            #Como no hay consistencia de ciclo no neecesitamos entrenar los segundos generadores 
            #Lo juntamos todo, no nos sirve para entreno solo para ver como va la cosa
            D_loss = disc_adult_down_loss
            
            #adults_reals += disc_adult_down_real.mean().item()
            #adults_fakes += disc_adult_down_fake.mean().item()
        
        
        #APLICAMOS LOS CAMBIOS DEL ENTRENO:
        
        #DOWN
        opt_disc_Adults_down.zero_grad()
        d_down_scaler.scale(D_loss).backward()
        d_down_scaler.step(opt_disc_Adults_down)
        d_down_scaler.update()
        
        
        #Ahora entrenamos los generadores con lo que hemos sacado de los discriminadores
        with torch.cuda.amp.autocast():
            generated_images = down_sampler(kid)
            #adversarial loss para los generadores
            dis_adult_down_fake = disc_Adults_down(generated_images)
            #loss_gen_down_adult = mse(dis_adult_down_fake, torch.ones_like(dis_adult_down_fake))
            loss_gen_down_adult = torch.mean(dis_adult_down_fake)
            writer.add_scalar("GenLoss", loss_gen_down_adult, i)

            
            #identity loss (para no alterar las caracteristicas principales)
            identity_adult_down = down_sampler(adult)
            identity_adult_down_loss = l1(adult_frontier, identity_adult_down)
            writer.add_scalar("IdentityLoss", identity_adult_down_loss, i)
            
            #Se suma todo
            g_down_loss = (loss_gen_down_adult + identity_adult_down_loss*LAMBDA_IDENTITY)
            
        #APLICAMOS LOS CAMBIOS DEL ENTRENO:
        
        #DOWN
        opt_gen_down.zero_grad()
        g_down_scaler.scale(g_down_loss).backward()
        g_down_scaler.step(opt_gen_down)
        g_down_scaler.update()

        
        if idx%500==0:
            save_image(generated_images*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/output{idx}.png")
            save_image(kid*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/input{idx}.png")
            save_image(adult_frontier*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/target{idx}.png")
            save_checkpoint(down_sampler, opt_gen_down, filename="/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/Modelos/gen_down.pth.tar")
            save_checkpoint(disc_Kids_down, opt_disc_Adults_down, filename="/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/Modelos/disc_Kids_down.pth.tar")
            save_checkpoint(disc_Adults_down, opt_disc_Kids_down, filename="/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/Modelos/disc_Adults_down.pth.tar")

            
            img_grid = torchvision.utils.make_grid(adult)
            #matplotlib_imshow(img_grid, one_channel=True)

            writer.add_image('input_down', torchvision.utils.make_grid(kid), i)

            writer.add_image('output_down', torchvision.utils.make_grid(fake_adult_down), i)

            writer.add_image('target', torchvision.utils.make_grid(adult_frontier), i)
            #writer.add_image('fake_adults_down', torchvision.utils.make_grid(fake_adult_down),idx)
            #writer.add_image('fake_adults_res', torchvision.utils.make_grid(fake_adult_res), idx)
            #writer.add_graph(down_sampler,kid)
        
        loop.set_postfix(GenLoss = torch.mean(g_down_loss), DisLoss = torch.mean(D_loss))
            
            

In [ ]:
[0.5 for _ in range(3)]

[0.5, 0.5, 0.5]

In [ ]:
def main():
    
    
    #Dataset
    img_size = 128
    frontier = 64
    channels = 3
    
    dataset_full = A.Compose([
        A.Resize(width=img_size, height=img_size),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5 for _ in range(channels)], std=[0.5 for _ in range(channels)], max_pixel_value=255),
        ToTensorV2(),
    ],additional_targets={"image0": "image"},)

    dataset_frontier = A.Compose([
            A.Resize(width=frontier, height=frontier),
            A.HorizontalFlip(p=0.5),
            A.Normalize(mean=[0.5 for _ in range(channels)], std=[0.5 for _ in range(channels)], max_pixel_value=255),
            ToTensorV2(),
        ],additional_targets={"image0": "image"},)
    
        
    train_path = "."
    Adults_train = os.path.join(train_path, 'Kids')
    Kids_train = os.path.join(train_path, 'Adults')
    
    dataset = TFGAN_Dataset(Adults_train, Kids_train, transform1=dataset_full, transform2=dataset_frontier,gray_scale=False)
    loader = DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=NUM_WORKERS,pin_memory=True,)
    
    #Loss functions
    L1 = nn.L1Loss()
    mse = nn.MSELoss()
    
    #DOWN
    """disc_Adults_down = Discriminador(frontier,channels=channels,frontier=4).to(DEVICE)
    disc_Kids_down = Discriminador(frontier,channels=channels,frontier=4).to(DEVICE)
    down_sampler = Generador_down(img_size,init_channels=channels,frontier=frontier, paso_de_canal=5, rabit_hole=3, num_residuals=2).to(DEVICE)
    opt_disc_Adults_down = optim.Adam(disc_Adults_down.parameters(),
                               lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
    opt_disc_Kids_down = optim.Adam(disc_Kids_down.parameters(),
                               lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
    opt_gen_down = optim.Adam(down_sampler.parameters(),
                              lr=LEARNING_RATE_GEN,betas=(0.5, 0.999),)
    d_down_scaler = torch.cuda.amp.GradScaler()
    g_down_scaler = torch.cuda.amp.GradScaler()"""

    disc_Adults_down = Discriminador(frontier,channels=channels,frontier=4).to(DEVICE)
    disc_Kids_down = Discriminador(frontier,channels=channels,frontier=4).to(DEVICE)
    down_sampler = Generador_down(img_size,init_channels=channels,frontier=frontier, paso_de_canal=5, rabit_hole=3, num_residuals=2).to(DEVICE)
    opt_disc_Adults_down = optim.Adam(disc_Adults_down.parameters(),
                               lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
    opt_disc_Kids_down = optim.Adam(disc_Kids_down.parameters(),
                               lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
    opt_gen_down = optim.Adam(down_sampler.parameters(),
                              lr=LEARNING_RATE_GEN,betas=(0.5, 0.999),)
    d_down_scaler = torch.cuda.amp.GradScaler()
    g_down_scaler = torch.cuda.amp.GradScaler()

    #carga
    """load_checkpoint("/content/drive/MyDrive/TFG/Images/Entrenamiento9/dis_Adults.tar", disc_Adults_down, opt_disc_Adults_down, LEARNING_RATE_DIS)
    load_checkpoint("/content/drive/MyDrive/TFG/Images/Entrenamiento9/disc_Kids.pth.tar", disc_Kids_down, opt_disc_Kids_down, LEARNING_RATE_DIS)
    load_checkpoint("/content/drive/MyDrive/TFG/Images/Entrenamiento9/gen_down.pth.tar", down_sampler, opt_gen_down, LEARNING_RATE_GEN)"""


    
    #RES
    disc_Adults_res = Discriminador(frontier,channels=channels, frontier=8).to(DEVICE)
    disc_Kids_res = Discriminador(frontier,channels=channels, frontier=8).to(DEVICE)
    res_sampler = Generador_res(frontier,4,num_de_paso=3,init_channels=channels,paso_de_canal=5,num_residuals=4).to(DEVICE)
    opt_disc_res = optim.Adam(list(disc_Adults_res.parameters()) + list(disc_Kids_res.parameters()),
                               lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
    opt_gen_res = optim.Adam(list(res_sampler.parameters()),
                              lr=LEARNING_RATE_GEN,betas=(0.5, 0.999),)
    d_res_scaler = torch.cuda.amp.GradScaler()
    g_res_scaler = torch.cuda.amp.GradScaler()
    
    
    #UP
    disc_Adults_up = Discriminador(img_size,channels=channels,frontier=8).to(DEVICE)
    disc_Kids_up = Discriminador(img_size,channels=channels,frontier=8).to(DEVICE)
    up_sampler = UpSampler(img_size, channels=channels,frontier=frontier).to(DEVICE)
    opt_disc_up = optim.Adam(list(disc_Adults_up.parameters()) + list(disc_Kids_up.parameters()),
                               lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
    opt_gen_up = optim.Adam(list(up_sampler.parameters()),
                              lr=LEARNING_RATE_GEN,betas=(0.5, 0.999),)
    d_up_scaler = torch.cuda.amp.GradScaler()
    g_up_scaler = torch.cuda.amp.GradScaler()
    
    #debug
    print(down_sampler)
    x = torch.randn((1,channels,img_size,img_size)).to(DEVICE)
    print(down_sampler(x).shape)
    #TensorBoard
    writer = SummaryWriter()
    i = 0
    for epoch in range(NUM_EPOCHS):
        """train_individual(
            writer, epoch, i,
            loader, L1, mse,
            disc_Adults_down, disc_Kids_down, down_sampler, opt_disc_down, opt_gen_down, d_down_scaler, g_down_scaler,
            disc_Adults_res, disc_Kids_res, res_sampler, opt_disc_res, opt_gen_res, d_res_scaler, g_res_scaler,
            disc_Adults_up, disc_Kids_up, up_sampler, opt_disc_up, opt_gen_up, d_up_scaler, g_up_scaler
        
        )"""

        print(f"Currently in epoch: {epoch+1}")
        writer.add_scalar("Epoch", epoch, i)
        train_individual_down(
            writer, epoch, i,
            loader, L1, mse,
            disc_Adults_down, disc_Kids_down, down_sampler, 
            opt_disc_Adults_down, opt_disc_Kids_down, 
            opt_gen_down, d_down_scaler, g_down_scaler,
        )
        i+=1
    writer.close()
        
    return

if __name__ == "__main__":
    main()

Generador_down(
  (model_sequential): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (5): ReLU(inplace=True)
    (6): ResidualBlock(
      (block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
        (6): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
    )
    (7): ResidualBlock(
      (block): Seque

  0%|          | 0/658 [00:00<?, ?it/s]COMET ERROR: Could not convert image_data into an image; ignored
COMET ERROR: Could not convert image_data into an image; ignored
COMET ERROR: Could not convert image_data into an image; ignored
  2%|▏         | 12/658 [00:02<02:21,  4.58it/s, DisLoss=tensor(-0.0610, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>), GenLoss=tensor(0.1230, device='cuda:0', grad_fn=<MeanBackward0>)]


KeyboardInterrupt: ignored

In [ ]:
import torch
import sys
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image

#Dataset
img_size = 128
frontier = 64
channels = 3

dataset_full = A.Compose([
    A.Resize(width=img_size, height=img_size),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=[0.5 for _ in range(channels)], std=[0.5 for _ in range(channels)], max_pixel_value=255),
    ToTensorV2(),
],additional_targets={"image0": "image"},)

dataset_frontier = A.Compose([
        A.Resize(width=frontier, height=frontier),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5 for _ in range(channels)], std=[0.5 for _ in range(channels)], max_pixel_value=255),
        ToTensorV2(),
    ],additional_targets={"image0": "image"},)

    
train_path = "."
Adults_train = os.path.join(train_path, 'Kids')
Kids_train = os.path.join(train_path, 'Adults')

dataset = TFGAN_Dataset(Adults_train, Kids_train, transform1=dataset_full, transform2=dataset_frontier,gray_scale=False)
loader = DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=NUM_WORKERS,pin_memory=True,)
disc_Adults_down = Discriminador(frontier,channels=channels,frontier=4).to(DEVICE)
disc_Kids_down = Discriminador(frontier,channels=channels,frontier=4).to(DEVICE)
down_sampler = Generador_down(img_size,init_channels=channels,frontier=frontier, paso_de_canal=5, rabit_hole=3, num_residuals=2).to(DEVICE)
opt_disc_Adults_down = optim.Adam(disc_Adults_down.parameters(),
                            lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
opt_disc_Kids_down = optim.Adam(disc_Kids_down.parameters(),
                            lr=LEARNING_RATE_DIS,betas=(0.5, 0.999),)
opt_gen_down = optim.Adam(down_sampler.parameters(),
                          lr=LEARNING_RATE_GEN,betas=(0.5, 0.999),)
d_down_scaler = torch.cuda.amp.GradScaler()
g_down_scaler = torch.cuda.amp.GradScaler()

#carga
load_checkpoint("/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/Modelos/disc_Adults_down.pth.tar", disc_Adults_down, opt_disc_Adults_down, LEARNING_RATE_DIS)
load_checkpoint("/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/Modelos/disc_Kids_down.pth.tar", disc_Kids_down, opt_disc_Kids_down, LEARNING_RATE_DIS)
load_checkpoint("/content/drive/MyDrive/TFG/Images/EntrenamientoFINAL/Modelos/gen_down.pth.tar", down_sampler, opt_gen_down, LEARNING_RATE_GEN)

def genera_down(
    writer, epoch, i,
    loader, l1, mse, 
    disc_Adults_down, disc_Kids_down, down_sampler, opt_disc_Adults_down, opt_disc_Kids_down, opt_gen_down, d_down_scaler, g_down_scaler,
):
    
    adults_reals = 0
    adults_fakes = 0
    
    loop = tqdm(loader, leave=True)
    
    for idx, (adult, kid, adult_frontier, kid_frontier) in enumerate(loop):
      with torch.cuda.amp.autocast():
        generated_images = down_sampler(kid)
        save_image(generated_images*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/Resultados/output{idx}.png")
        save_image(kid*0.5+0.5,f"/content/drive/MyDrive/TFG/Images/Resultados/input{idx}.png")
      



genera_down(
            None, None, 0,
            loader, None, None,
            disc_Adults_down.to(DEVICE), disc_Kids_down.to(DEVICE), down_sampler.to(DEVICE), 
            opt_disc_Adults_down, opt_disc_Kids_down, 
            opt_gen_down, d_down_scaler, g_down_scaler,
        )
            

=> Loading checkpoint
=> Loading checkpoint
=> Loading checkpoint


  0%|          | 0/658 [00:00<?, ?it/s]


RuntimeError: ignored